3124













01# 第 10 章 数据聚合与分组运算         P303

## 10.1 GroupBy 机制   P303

In [5]:
import pandas as pd
import numpy as np

In [ ]:
#第一个阶段，pandas 对 象(无论是 Series、DataFrame 还是其他的)中的数据会根据你所提供的一个 或多个键被拆分(split)为多组。
#拆分操作是在对象的特定轴上执行的。例如，DataFrame 可以在其行(axis=0)或列(axis=1)上进行分组。
#然后，将 一个函数应用(apply)到各个分组并产生一个新值。最后，所有这些函数的执 行结果会被合并(combine)到最终的结果对象中。
#结果对象的形式一般取决于 数据上所执行的操作。图 10-1 大致说明了一个简单的分组聚合过程。

In [6]:
#首先来看看下面这个非常简单的表格型数据集(以 DataFrame 的形式):
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.759831,-1.587621
1,a,two,-0.449592,1.016323
2,b,one,1.842260,0.012293
3,b,two,1.091384,0.364485
4,a,one,0.202038,2.438704


In [9]:
#假设你想要按 key1 进行分组，并计算 data1 列的平均值。
#实现该功能的方式有 很多，而我们这里要用的是:访问 data1，并根据 key1 调用 groupby:
grouped = df['data1'].groupby(df['key1'])
grouped

In [10]:
#变量 grouped 是一个 GroupBy 对象。它实际上还没有进行任何计算，只是含有 一些有关分组键 df['key1']的中间数据而已。
#换句话说，该对象已经有了接下 来对各分组执行运算所需的一切信息。例如，我们可以调用 GroupBy 的 mean 方 法来计算分组平均值:
grouped.mean()

key1
a   -0.335795
b    1.466822
Name: data1, dtype: float64

In [11]:
#如果我们一次传入多个数组的列表，就会得到不同的结果:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.278897
      two    -0.449592
b     one     1.842260
      two     1.091384
Name: data1, dtype: float64

In [12]:
#这里，我通过两个键对数据进行了分组，得到的 Series 具有一个层次化索引 (由唯一的键对组成):
means.unstack()

key2,one,two
key1,,
a,-0.278897,-0.449592
b,1.842260,1.091384


In [ ]:
#在这个例子中，分组键均为 Series。实际上，分组键可以是任何长度适当的数 组:

In [13]:
states = np.array(['Ohio', 'California', 'California',
'Ohio', 'Ohio'])
states

array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'], dtype='<U10')

In [14]:
years = np.array([2005, 2005, 2006, 2005, 2006])
years

array([2005, 2005, 2006, 2005, 2006])

In [15]:
df['data1']

0   -0.759831
1   -0.449592
2    1.842260
3    1.091384
4    0.202038
Name: data1, dtype: float64

In [19]:
df['data1'].groupby([states, years]).mean()

California  2005   -0.449592
            2006    1.842260
Ohio        2005    0.165776
            2006    0.202038
Name: data1, dtype: float64

In [21]:
#分组信息就位于相同的要处理 DataFrame 中。这里，你还可以将列名 (可以是字符串、数字或其他 Python 对象)用作分组键:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.335795,0.622469
b,1.466822,0.188389


In [22]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.278897  0.425541
     two  -0.449592  1.016323
b    one   1.842260  0.012293
     two   1.091384  0.364485

In [ ]:
#无论你准备拿 groupby 做什么，都有可能会用到 GroupBy 的 size 方法，它可以 返回一个含有分组大小的 Series:

In [23]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [ ]:
#注意，任何分组关键词中的缺失值，都会被从结果中除去。

## 对分组进行迭代

In [28]:
# GroupBy 对象支持迭代，可以产生一组二元元组(由分组名和数据块组成)。 看下面的例子:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.759831 -1.587621
1    a  two -0.449592  1.016323
4    a  one  0.202038  2.438704
b
  key1 key2     data1     data2
2    b  one  1.842260  0.012293
3    b  two  1.091384  0.364485


In [29]:
# 对于多重键的情况，元组的第一个元素将会是由键值组成的元组:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.759831 -1.587621
4    a  one  0.202038  2.438704
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.449592  1.016323
('b', 'one')
  key1 key2    data1     data2
2    b  one  1.84226  0.012293
('b', 'two')
  key1 key2     data1     data2
3    b  two  1.091384  0.364485


In [30]:
#你可以对这些数据片段做任何操作。有一个你可能会觉得有用的运算:将这些数据片段做成一个字典:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one -0.759831 -1.587621
 1    a  two -0.449592  1.016323
 4    a  one  0.202038  2.438704, 'b':   key1 key2     data1     data2
 2    b  one  1.842260  0.012293
 3    b  two  1.091384  0.364485}

In [32]:
list(df.groupby('key1'))

[('a',   key1 key2     data1     data2
  0    a  one -0.759831 -1.587621
  1    a  two -0.449592  1.016323
  4    a  one  0.202038  2.438704), ('b',   key1 key2     data1     data2
  2    b  one  1.842260  0.012293
  3    b  two  1.091384  0.364485)]

In [33]:
pieces['b']

,key1,key2,data1,data2
2,b,one,1.842260,0.012293
3,b,two,1.091384,0.364485


In [34]:
#groupby 默认是在 axis=0 上进行分组的，通过设置也可以在其他任何轴上进行 分组。
#拿上面例子中的 df 来说，我们可以根据 dtype 对列进行分组:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [40]:
grouped = df.groupby(df.dtypes, axis=1)

In [43]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.759831 -1.587621
1 -0.449592  1.016323
2  1.842260  0.012293
3  1.091384  0.364485
4  0.202038  2.438704
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## 选取一列或列的子集

In [ ]:
对于#由 DataFrame 产生的 GroupBy 对象，如果用一个(单个字符串)或一组 (字符串数组)列名对其进行索引，就能实现选取部分列进行聚合的目的。
#也 就是说:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]

In [ ]:
#是以下代码的语法糖:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [48]:
df.groupby('key1')['data1'].mean()

key1
a   -0.335795
b    1.466822
Name: data1, dtype: float64

In [49]:
df['data1'].groupby(df['key1']).mean()

key1
a   -0.335795
b    1.466822
Name: data1, dtype: float64

In [50]:
#尤其对于大数据集，很可能只需要对部分列进行聚合。
#例如，在前面那个数据 集中，如果只需计算 data2 列的平均值并以 DataFrame 形式得到结果，可以这 样写:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.425541
     two   1.016323
b    one   0.012293
     two   0.364485

In [51]:
#这种索引操作所返回的对象是一个已分组的 DataFrame(如果传入的是列表或 数组)或已分组的 Series(如果传入的是标量形式的单个列名):
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped.

In [52]:
s_grouped.mean()

key1  key2
a     one     0.425541
      two     1.016323
b     one     0.012293
      two     0.364485
Name: data2, dtype: float64

## 通过字典或 Series 进行分组

In [53]:
# 除数组以外，分组信息还可以其他形式存在。来看另一个示例 DataFrame:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,0.408691,0.730999,0.719931,-1.001531,-1.195295
Steve,1.484426,-0.222744,0.438632,-0.881987,0.830169
Wes,0.385294,0.420900,0.706069,-0.118025,-0.473310
Jim,0.347850,-0.095909,0.937391,-1.258321,0.490775
Travis,0.516145,0.258516,-0.689073,0.186034,-0.251191


In [54]:
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values、
people

,a,b,c,d,e
Joe,0.408691,0.730999,0.719931,-1.001531,-1.195295
Steve,1.484426,-0.222744,0.438632,-0.881987,0.830169
Wes,0.385294,NaN,NaN,-0.118025,-0.473310
Jim,0.347850,-0.095909,0.937391,-1.258321,0.490775
Travis,0.516145,0.258516,-0.689073,0.186034,-0.251191


In [55]:
# 假设已知列的分组关系，并希望根据分组计算列的和:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}
mapping

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [56]:
#你可以将这个字典传给 groupby，来构造数组，但我们可以直接传递字 典(我包含了键“f”来强调，存在未使用的分组键是可以的):
by_column = people.groupby(mapping, axis=1)
by_column

In [59]:
by_column.sum()

,blue,red
Joe,-0.281600,-0.055605
Steve,-0.443355,2.091851
Wes,-0.118025,-0.088016
Jim,-0.320930,0.742716
Travis,-0.503039,0.523470


In [60]:
#Series 也有同样的功能，它可以被看做一个固定大小的映射:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [61]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## 通过函数进行分组 

In [ ]:
#使用 Python 函数是一种更原生的方法定义分组映 射。任何被当做分组键的函数都会在各个索引值上被调用一次，其返回值就会 被用作分组名称。

In [62]:
#以上一小节的示例 DataFrame 为例，其索引值为 人的名字。你可以计算一个字符串长度的数组，更简单的方法是传入 len 函 数:
people

,a,b,c,d,e
Joe,0.408691,0.730999,0.719931,-1.001531,-1.195295
Steve,1.484426,-0.222744,0.438632,-0.881987,0.830169
Wes,0.385294,NaN,NaN,-0.118025,-0.473310
Jim,0.347850,-0.095909,0.937391,-1.258321,0.490775
Travis,0.516145,0.258516,-0.689073,0.186034,-0.251191


In [63]:
people.groupby(len)

In [64]:
people.groupby(len).sum()  #len()代表字符串长度

,a,b,c,d,e
3,1.141834,0.635090,1.657322,-2.377878,-1.177829
5,1.484426,-0.222744,0.438632,-0.881987,0.830169
6,0.516145,0.258516,-0.689073,0.186034,-0.251191


In [66]:
#将函数跟数组、列表、字典、Series 混合使用也不是问题，因为任何东西在内 部都会被转换为数组:
key_list = ['one', 'one', 'one', 'two', 'two']

In [67]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  0.385294  0.730999  0.719931 -1.001531 -1.195295
  two  0.347850 -0.095909  0.937391 -1.258321  0.490775
5 one  1.484426 -0.222744  0.438632 -0.881987  0.830169
6 two  0.516145  0.258516 -0.689073  0.186034 -0.251191

## 根据索引级别分组

In [68]:
#层次化索引数据集最方便的地方就在于它能够根据轴索引的一个级别进行聚合:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US',
'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
columns

MultiIndex(levels=[['JP', 'US'], [1, 3, 5]],
           codes=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=['cty', 'tenor'])

In [69]:
hier_df = pd.DataFrame(np.random.randn(4, 5),
columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.194005  0.108622  0.012292 -1.484075  0.701317
1     -0.512341  0.263893 -0.596110  0.204424  0.824680
2      0.433250 -0.715658  0.436834 -0.659589 -1.925740
3      0.679417  0.022365  0.093923  1.639215 -0.930853

In [70]:
#要根据级别分组，使用 level 关键字传递级别序号或名字:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 数据聚合  P313

In [ ]:
#聚合指的是任何能够从数组产生标量值的数据转换过程。

In [ ]:
#GroupBy 会高效地对 Series 进行切片，然后对各片调 用 piece.quantile(0.9)，最后将这些结果组装成最终结果:

In [71]:
df

,key1,key2,data1,data2
0,a,one,-0.759831,-1.587621
1,a,two,-0.449592,1.016323
2,b,one,1.842260,0.012293
3,b,two,1.091384,0.364485
4,a,one,0.202038,2.438704


In [73]:
grouped = df.groupby('key1')
grouped

In [74]:
grouped['data1'].quantile(0.9)   #quantile 样本分位数

key1
a    0.071712
b    1.767173
Name: data1, dtype: float64

In [ ]:
#如果要使用你自己的聚合函数，只需将其传入 aggregate 或 agg 方法即可:

In [75]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [76]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.961869,4.026325
b,0.750877,0.352191


In [77]:
#有些方法(如 describe)也是可以用在这里的，即使严格 来讲，它们并非聚合运算:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.335795  0.490928 -0.759831 -0.604712 -0.449592 -0.123777   
b      2.0  1.466822  0.530950  1.091384  1.279103  1.466822  1.654541   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.202038   3.0  0.622469  2.041853 -1.587621 -0.285649  1.016323   
b     1.842260   2.0  0.188389  0.249037  0.012293  0.100341  0.188389   

                          
           75%       max  
key1                      
a     1.727513  2.438704  
b     0.276437  0.364485

## 面向列的多函数应用

In [78]:
#回到前面小费的例子。使用 read_csv 导入数据之后，我们添加了一个小费百分 比的列 tip_pct:
tips = pd.read_csv('examples/tips.csv')
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
5,25.29,4.71,No,Sun,Dinner,4
6,8.77,2.00,No,Sun,Dinner,2
7,26.88,3.12,No,Sun,Dinner,4
8,15.04,1.96,No,Sun,Dinner,2
9,14.78,3.23,No,Sun,Dinner,2


In [80]:
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240
6,8.77,2.00,No,Sun,Dinner,2,0.228050
7,26.88,3.12,No,Sun,Dinner,4,0.116071
8,15.04,1.96,No,Sun,Dinner,2,0.130319
9,14.78,3.23,No,Sun,Dinner,2,0.218539


In [81]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [ ]:
#对 Series 或 DataFrame 列的聚合运算其实就是使用 aggregate (使用自定义函数)或调用诸如 mean、std 之类的方法。

In [82]:
#你可能希望对 不同的列使用不同的聚合函数，或一次应用多个函数。
#其实这也好办，我将通 过一些示例来进行讲解。首先，我根据天和 smoker 对 tips 进行分组:
grouped = tips.groupby(['day', 'smoker'])

In [84]:
grouped_pct = grouped['tip_pct']
grouped_pct

In [85]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [86]:
#如果传入一组函数或函数名，得到的 DataFrame 的列就会以相应的函数命名:
grouped_pct.agg(['mean', 'std', peak_to_peak])   #mean 取平均素， std 取标准差， peak_to_peak上面定义的函数 最大值减去最小值

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [ ]:
#你并非一定要接受 GroupBy 自动给出的那些列名，特别是 lambda 函数，它们的 名称是'<lambda>'，这样的辨识度就很低了(通过函数的 name 属性看看就知道 了)。
#因此，如果传入的是一个由(name,function)元组组成的列表，则各元组 的第一个元素就会被用作 DataFrame 的列名(可以将这种二元元组列表看做一 个有序映射):

In [87]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [ ]:
#对于 DataFrame，你还有更多选择，你可以定义一组应用于全部列的一组函 数，或不同的列应用不同的函数。假设我们想要对 tip_pct 和 total_bill 列计 算三个统计信息:

In [89]:
functions = ['count', 'mean', 'max']

In [90]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

In [91]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [92]:
#结果 DataFrame 拥有层次化的列，这相当于分别对各列进行聚合， 然后用 concat 将结果组装到一起，使用列名用作 keys 参数:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [93]:
#这里也可以传入带有自定义名称的一组元组:
ftuples = [('Durchschnitt', 'mean'),('Abweichung', np.var)]

In [94]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [95]:
#假设你想要对一个列或不同的列应用不同的函数。具体的办法是向 agg 传入一个从列名映射到函数的字典:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [96]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

## 以“没有行索引”的形式返回聚合数据

In [ ]:
#你可以向 groupby 传入 as_index=False 以禁用该功能:

In [97]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [ ]:
#对结果调用 reset_index 也能得到这种形式的结果。

In [106]:
test = tips.groupby(['day', 'smoker']).mean()
test.reset_index

<bound method DataFrame.reset_index of              total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863>

## 10.3 apply:一般性的“拆分-应用-合并”   P319

In [ ]:
#最通用的 GroupBy 方法是 apply，apply 会将待处理的对象拆分成多个片段，然后对各片段调用传入的函 数，最后尝试将各片段组合到一起。

In [107]:
#回到之前那个小费数据集，假设你想要根据分组选出最高的 5 个 tip_pct 值。
#首先，编写一个选取指定列具有最大值的行的函数:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [108]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


## 10.4 透视表和交叉表  P332

## 10.5 总结  P336